In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/D4D')

In [ ]:
!pip install warmup_scheduler
!pip install wandb

In [ ]:
from trainer import Trainer, ConfigParser
from model import NoiseEstimationClip, NoiseEstimationCLIP_pretrained
from dataset import NoiseEstimationDataset, NoiseEstimationValidationDataset, create_dataloaders
from types import SimpleNamespace

In [ ]:
def flatten_namespace(nested_namespace):
    flat_namespace = SimpleNamespace()
    def add_attributes(ns):
        for key, value in vars(ns).items():
            if isinstance(value, SimpleNamespace):
                add_attributes(value)
            else:
                setattr(flat_namespace, key, value)

    add_attributes(nested_namespace)
    return flat_namespace

In [ ]:
config = ConfigParser.parse_yaml('/content/drive/MyDrive/D4D/config.yaml')
config = flatten_namespace(config)

In [ ]:
dataset = NoiseEstimationDataset(image_dir=config.valid_dir,
                                  clean_image=config.image_dir,
                                  img_size=config.image_size,
                                  specific_timesteps=config.specific_timesteps,
                                  saved_all_data_first=True,
                                  num_cores=config.num_cores)

Using device: cuda


/content/drive/MyDrive/D4D/utils/DDPMForward.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x_0 = torch.load(clean_image)


In [ ]:
valid_dataset = NoiseEstimationValidationDataset(image_dir=config.valid_dir,
                                        clean_image=config.valid_image,
                                        img_size=config.image_size,
                                        specific_timesteps=config.specific_timesteps,
                                        saved_all_data_first=True,
                                        num_cores=config.num_cores)
print(valid_dataset.specified_timesteps)

Using device: cuda
80


In [ ]:
dataloader = create_dataloaders(dataset,
                                batch_size=config.batch_size,
                                num_workers=config.num_workers)

In [ ]:
valid_dataloader = create_dataloaders(valid_dataset,
                                      batch_size=config.batch_size,
                                      num_workers=config.num_workers)

In [ ]:
if config.text_image:
  print('Using text image model')
  model = NoiseEstimationCLIP_pretrained(model_name=config.clip_pretrained_model_name)
else:
  model = NoiseEstimationClip(d_model=config.d_model,
                              in_channels=config.in_channels,
                              image_size=config.image_size,
                              patch_size=config.patch_size,
                              num_heads=config.num_heads,
                              num_layers=config.num_layers,
                              final_embedding=config.final_embedding_dim)

Using text image model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
trainer = Trainer(model, dataloader, valid_dataloader, config)

Using device: cuda


/content/drive/MyDrive/D4D/trainer.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)


In [ ]:
trainer.train()